In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from lxml import html
import datetime
import time
from selenium.common.exceptions import (NoSuchElementException,ElementClickInterceptedException,ElementNotInteractableException,NoSuchAttributeException,ElementNotVisibleException,TimeoutException)
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [13]:
url = "https://georgiaprobaterecords.com/Estates/SearchEstates.aspx"

#options = webdriver.ChromeOptions()
#options.add_argument('headless')
#driver = webdriver.Chrome(executable_path='C:/Users/Admin/Documents/Python/chromedriver.exe',options=options)

driver = webdriver.Chrome(executable_path='C:/Users/Admin/Documents/Python/chromedriver.exe')

countylist = ['Carroll','Fayette','Clayton','Coweta','Paulding','Douglas','Jasper','Fannin','Hall','Pickens','Barrow','Newton','Meriwether']
#countylist = ['Meriwether']
column_names = ['County','Petitioner_First_Name','Petitioner_Middle_Name','Petitioner_Last_Name','Decedent_Street','Decedent_City','Decedent_State','Decedent_ZIP','Petition_Link_1','Petition_Link_2','Petition_Link_3']
df = pd.DataFrame(columns=column_names)
ind = 0

#EDIT ME
startdate = '2022-09-28'

for c in countylist:
    driver.get(url)
    driver.find_element("xpath","/html/body/form/div[3]/div[2]/div[2]/div[1]/table/tbody/tr[1]/td[2]/div/div/span/span[1]").click()
    for i in range(0,74):
        if driver.find_element("xpath",f"/html/body/form/div[1]/div/ul/li[{i+1}]").text == c:
            driver.find_element("xpath",f"/html/body/form/div[1]/div/ul/li[{i+1}]").click()
            driver.find_element("xpath","//*[@id=\"ctl00_cpMain_txtFiledStartDate_dateInput\"]").send_keys(startdate)
            driver.find_element("xpath","//*[@id=\"ctl00_cpMain_txtFiledEndDate_dateInput\"]").send_keys(str(datetime.date.today()))
            driver.find_element("xpath","//*[@id=\"ctl00_cpMain_btnSearch_input\"]").click()
            WebDriverWait(driver, 8).until(EC.presence_of_element_located((By.XPATH, "//*[@id=\"ctl00_cpMain_rgEstates_ctl00__0\"]/td[1]/a")))
            try: 
                pr = int(driver.find_element("xpath","//*[@id=\"ctl00_cpMain_rgEstates_ctl00\"]/tfoot/tr/td/table/tbody/tr/td/div[5]/strong[2]").text)
            except NoSuchElementException:
                pr = 1
            for k in range(0,pr):    
            #for k in range(1,2):    
                for j in range(0,10):
                #for j in range(1,2)
                    try:
                        driver.switch_to.window(driver.window_handles[0])
                        element = driver.find_element("xpath",f"//*[@id=\"ctl00_cpMain_rgEstates_ctl00__{j}\"]/td[1]/a")
                        driver.execute_script(f"window.open('{element.get_attribute('href')}', 'new_window')")
                        driver.switch_to.window(driver.window_handles[1])
                        try:
                            for p in range(0,20):
                                try:
                                    if 'pet' in driver.find_element("xpath",f"//*[@id=\"cpMain_repParty_lblPartyType_{p}\"]").text.lower():
                                        pn = driver.find_element("xpath",f"//*[@id=\"cpMain_repParty_lblParty_{p}\"]").text
                                        break
                                    elif 'admin' in driver.find_element("xpath",f"//*[@id=\"cpMain_repParty_lblPartyType_{p}\"]").text.lower():
                                        pn = driver.find_element("xpath",f"//*[@id=\"cpMain_repParty_lblParty_{p}\"]").text
                                        break
                                    elif 'exec' in driver.find_element("xpath",f"//*[@id=\"cpMain_repParty_lblPartyType_{p}\"]").text.lower():
                                        pn = driver.find_element("xpath",f"//*[@id=\"cpMain_repParty_lblParty_{p}\"]").text
                                        break
                                    elif 'guard' in driver.find_element("xpath",f"//*[@id=\"cpMain_repParty_lblPartyType_{p}\"]").text.lower():
                                        pn = driver.find_element("xpath",f"//*[@id=\"cpMain_repParty_lblParty_{p}\"]").text
                                        break
                                    elif 'heir' in driver.find_element("xpath",f"//*[@id=\"cpMain_repParty_lblRelationship_{p}\"]").text.lower() and 'dod' not in driver.find_element("xpath",f"//*[@id=\"cpMain_repParty_lblRelationship_{p}\"]").text.lower() and 'estate' not in driver.find_element("xpath",f"//*[@id=\"cpMain_repParty_lblRelationship_{p}\"]").text.lower():
                                        pn = driver.find_element("xpath",f"//*[@id=\"cpMain_repParty_lblParty_{p}\"]").text
                                        break
                                except NoSuchElementException:
                                    pn = ""
                                    pass
                            if pn == "":
                                try:
                                    pn = driver.find_element("xpath",f"//*[@id=\"cpMain_repParty_lblParty_0\"]").text
                                except NoSuchElementException:
                                    pn = ""
                                    pass
                        except NoSuchElementException:
                            pn = ""
                            pass
                        if pn == "":
                            pfn = ""
                            pmn = ""
                            pln = ""
                        elif len(pn.split(" ")) == 1:
                            pfn = pn.split(" ")[0]
                            pmn = ""
                            pln = ""
                        elif len(pn.split(" ")) == 2:
                            pfn = pn.split(" ")[0]
                            pmn = ""
                            pln = pn.split(" ")[1]
                        elif len(pn.split(" ")) >= 3:
                            pfn = pn.split(" ")[0]
                            pmn = pn.split(" ")[1]
                            pln = pn.split(" ")[2]
                        try:
                            ds = driver.find_element("xpath","//*[@id=\"cpMain_lblStreetAddress\"]").text
                        except NoSuchElementException:
                            ds = ""
                            pass
                        try:
                            dcsz = driver.find_element("xpath","//*[@id=\"cpMain_lblCityStateZip\"]").text
                        except NoSuchElementException:
                            dcsz = ""
                            pass
                        if dcsz == "":
                            dc = ""
                            dst = ""
                            dz = ""
                        else:
                            if dcsz.split(',')[1][0] == " ":
                                dc = dcsz.split(',')[0]
                                dst = dcsz.split(',')[1].split(' ')[1]
                                dz = dcsz.split(',')[1].split(' ')[2]
                            else:
                                dc = dcsz.split(',')[0]
                                dst = dcsz.split(',')[1].split(' ')[0]
                                dz = dcsz.split(',')[1].split(' ')[1]
                        linklist = []
                        try:
                            driver.find_element("xpath","//*[@id=\"cpMain_Label3\"]")
                            for f in range(0,30):
                                if driver.find_element("xpath",f"//*[@id=\"cpMain_repDocuments_lblDocumentName_{f}\"]").text[0:3].lower() == 'pet':
                                    linklist.append(driver.find_element("xpath",f"//*[@id=\"cpMain_repDocuments_lnkView_{f}\"]").get_attribute('href'))
                                    print('Files')
                            continue
                        except NoSuchElementException:
                            pass
                        try:
                            driver.find_element("xpath","//*[@id=\"cpMain_Label2\"]")
                            for f2 in range(0,30):
                                try:
                                    if driver.find_element("xpath",f"//*[@id=\"cpMain_repFilings_repFilingDetails_0_lblProbateEventDesc_{f2}\"]").text[0:3].lower() == 'pet':
                                        linklist.append(driver.find_element("xpath",f"//*[@id=\"cpMain_repFilings_repFilingDetails_0_lnkView_{f2}\"]").get_attribute('href'))
                                        print('Files')
                                except NoSuchElementException:
                                    pass
                        except NoSuchElementException:
                            print('No Files')
                            pass
                        if len(linklist) == 0:
                            l1 = ""
                            l2 = ""
                            l3 = ""
                        elif len(linklist) == 1:
                            l1 = linklist[0]
                            l2 = ""
                            l3 = ""
                        elif len(linklist) == 2:
                            l1 = linklist[0]
                            l2 = linklist[1]
                            l3 = ""
                        elif len(linklist) >= 3:
                            l1 = linklist[0]
                            l2 = linklist[1]
                            l3 = linklist[2]
                        row = pd.DataFrame({'County':c,'Petitioner_First_Name':pfn,'Petitioner_Middle_Name':pmn,'Petitioner_Last_Name':pln,'Decedent_Street':ds,'Decedent_City':dc,'Decedent_State':dst,'Decedent_ZIP':dz,'Petition_Link_1':l1,'Petition_Link_2':l2,'Petition_Link_3':l3},index=[ind])
                        df = pd.concat([df,row],axis=0)
                        print(row.values)
                        ind += 1
                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])
                    except NoSuchElementException:
                        pass
                if int(k)+1 < pr:
                    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "//*[@id=\"ctl00_cpMain_rgEstates_ctl00\"]/tfoot/tr/td/table/tbody/tr/td/div[3]/input[1]")))
                    driver.find_element("xpath","//*[@id=\"ctl00_cpMain_rgEstates_ctl00\"]/tfoot/tr/td/table/tbody/tr/td/div[3]/input[1]").click()
            df.to_csv('C:/Users/Admin/Documents/Python/Sample.csv',index=False)
            break
    driver.switch_to.window(driver.window_handles[0])

C:\Users\Admin\AppData\Local\Temp\ipykernel_16952\52817073.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='C:/Users/Admin/Documents/Python/chromedriver.exe')


Files
[['Carroll' 'DANA' 'KRISTEN' 'RICHARDSON' '563 MADDOX ROAD' 'CARROLLTON'
  'GA' '30117'
  'https://georgiaprobaterecords.com/Imaging/ViewScannedImage.aspx?DocType=4&CaseID=OTQ1OTM2&CID=MTAyNg=='
  '' '']]
[['Carroll' 'ANGELA' 'JENKINS' 'GIBERSON' '3323 N HWY 27' 'CARROLLTON'
  'GA' '30117' '' '' '']]
Files
[['Carroll' 'KENNETH' 'FRANKLIN' 'BRITT' '580 DANIEL RD' 'VILLA RICA'
  'GA' '30180'
  'https://georgiaprobaterecords.com/Imaging/ViewScannedImage.aspx?DocType=4&CaseID=ODc3Mjgy&CID=MTAyNg=='
  '' '']]
[['Carroll' 'SHANNON' '' 'MORRIS' '' '' '' '' '' '' '']]
[['Carroll' 'SAMANTHA' 'L.' 'HENDRSON' '' '' '' '' '' '' '']]
[['Fayette' 'GERRI' 'YVONNE' 'NOTT' '1720 HWY 92S' 'FAYETTEVILLE' 'GA'
  '30215' '' '' '']]
Files
[['Fayette' 'CLAIRE' 'CRAIN' 'MONROE' '250 BROOKWOOD LANE' 'FAYETTEVILLE'
  'GA' '30215'
  'https://georgiaprobaterecords.com/Imaging/ViewScannedImage.aspx?DocType=4&CaseID=NjIxMTU1&CID=MTAyNA=='
  '' '']]
Files
[['Fayette' 'BRIAN' 'THOMAS' 'DUNGAN' '155 ELEANOR DRIV